## Un semplice esempio di uso dell'API di ChatGPT in modalità _speech-to-text_ e poi _text-to-speech_ 
Luca Mari, aprile 2023  
[virtenv `langchain`: openai, gtts, SpeechRecognition, pygame, pyaudio]

Qui l'esempio è configurato in modo da tradurre in inglese la frase pronunciata in italiano.

In [3]:
import os
from gtts import gTTS
import pygame
from io import BytesIO
import speech_recognition as sr
import openai

class Speech():
    def __init__(self) -> None:
        pygame.init()

    def listen(self, lang:str) -> str:
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("Parla pure...")
            audio_text = r.listen(source)
            print("Ok")
            try:
                return str(r.recognize_google(audio_text, language=lang))
            except:
                return "Errore..."

    def speak(self, text:str, lang:str) -> None:
        tts = gTTS(text=text, lang=lang)
        fp = BytesIO()
        tts.write_to_fp(fp)
        fp.seek(0)
        pygame.mixer.init()
        pygame.mixer.music.load(fp)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)

    def speak2(self, text:str, lang:str) -> None:
        myobj = gTTS(text=text, lang=lang, slow=False)
        myobj.save('temp.mp3')
        os.system('mpg123 temp.mp3')
        os.remove('temp.mp3')


class Dialogue():
    def __init__(self) -> None:
        openai.organization = os.getenv('OPENAI_ORGANIZATION')
        openai.api_key = os.getenv('OPENAI_API_KEY')

    def get_reply(self, prompt:str) -> str:
        messages = [{'role':'assistant', 'content':'Translate from Italian to English'}, {'role':'user', 'content':prompt}]
        completion = openai.ChatCompletion.create( # genera la risposta
            model='gpt-4',
            messages=messages,
            max_tokens=512,
            n=1,
            stop=None,
            temperature=0.5
        )
        return completion["choices"][0]["message"]["content"] # type: ignore

In [4]:

speech = Speech()
dialogue = Dialogue()
q_text = speech.listen('it')
print(q_text)
a_text = dialogue.get_reply(q_text)
speech.speak2(a_text, 'en')
print(a_text)

ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.0.pcm.iec958.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5703:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM iec958
ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.0.pcm.iec958.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5703:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM spdif
ALSA lib confmisc.c:1369:(snd_func_refer) Unable to find definition 'cards.0.pcm.iec958.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_refer returned error: No such file or direc

Parla pure...
Ok
non sono sicuro di capire perché ma il libro che sto leggendo non mi convince


High Performance MPEG 1.0/2.0/2.5 Audio Player for Layers 1, 2 and 3
	version 1.31.3; written and copyright by Michael Hipp and others
	free software (LGPL) without any warranty but with best wishes

Playing MPEG stream 1 of 1: temp.mp3 ...

MPEG 2.0 L III cbr32 24000 mono


I'm not sure why, but the book I'm reading doesn't convince me.



[0:04] Decoding of temp.mp3 finished.
